In [20]:
import tensorflow as tf
import keras as ks
# general
import os
import sys
import io

import argparse
from datetime import datetime
import h5py
import importlib

# Indispensables
import pandas as pd
import numpy as np
import geopandas as gpd
import math

# VISU
from IPython.display import Image, IFrame
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm


import colorsys
print('TensorFlow: %s' % tf.__version__)
print('Kerasw: %s' % ks.__version__)
print('Panda version: %s' % pd.__version__)
print('Numpy version: %s' % np.__version__)
print('Geo Panda version: %s' % gpd.__version__)


TensorFlow: 2.0.0
Kerasw: 2.3.1
Panda version: 0.25.1
Numpy version: 1.17.2
Geo Panda version: 0.7.0


In [21]:
BASE_DIR = os.getcwd()
# Eurométropole
DATA_DIR = "S:\\Commun\\SIG3D\\2020\\PROJETS_2020\\20039_MachineLearning\\DATA\\"
# Laptop
DATA_DIR = "/Users/pierreleisy/Data_Science/NOTEBOOK/STAGE/"
DALLE_NUM = DATA_DIR + "DallesNumPoints5m"
DALLE_ECA = DATA_DIR + "DallesEcartZ5m"
DALLE_POS = DATA_DIR + "DallesPosition5m"
print("Working directory:",BASE_DIR)
print("Data ROOT directory:",DATA_DIR)

sys.path.append(DATA_DIR)
sys.path.append(os.path.join(DATA_DIR,'DallesNumPoints5m\\'))
sys.path

Working directory: /Users/pierreleisy/C-PYTHON/training-Python/NOTEBOOK
Data ROOT directory: /Users/pierreleisy/Data_Science/NOTEBOOK/STAGE/


['/Users/pierreleisy/C-PYTHON/training-Python/NOTEBOOK',
 '/Users/pierreleisy/Data_Science/anaconda3/envs/geo_env/lib/python37.zip',
 '/Users/pierreleisy/Data_Science/anaconda3/envs/geo_env/lib/python3.7',
 '/Users/pierreleisy/Data_Science/anaconda3/envs/geo_env/lib/python3.7/lib-dynload',
 '',
 '/Users/pierreleisy/.local/lib/python3.7/site-packages',
 '/Users/pierreleisy/Data_Science/anaconda3/envs/geo_env/lib/python3.7/site-packages',
 '/Users/pierreleisy/Data_Science/anaconda3/envs/geo_env/lib/python3.7/site-packages/IPython/extensions',
 '/Users/pierreleisy/.ipython',
 '/Users/pierreleisy/Data_Science/NOTEBOOK/STAGE',
 '/Users/pierreleisy/Data_Science/NOTEBOOK/STAGE/DallesNumPoints5m\\',
 '/Users/pierreleisy/Data_Science/NOTEBOOK/STAGE/',
 '/Users/pierreleisy/Data_Science/NOTEBOOK/STAGE/DallesNumPoints5m\\',
 '/Users/pierreleisy/Data_Science/NOTEBOOK/STAGE/',
 '/Users/pierreleisy/Data_Science/NOTEBOOK/STAGE/DallesNumPoints5m\\']

In [22]:
from os import listdir
#from os.path import isfile, join
#from os import walk
import glob

#liste1 = [f for f in listdir(DALLE_NUM) if isfile(join(DALLE_NUM, f))]
liste_1 = [f for f in glob.glob(DALLE_NUM + "\\*.tif", recursive=True)][:2]   #only first 10 for test
liste_2 = [f for f in glob.glob(DALLE_ECA + "\\*.tif", recursive=True)][:2]  
liste_3 = [f for f in glob.glob(DALLE_POS + "\\*.tif", recursive=True)][:2]   
print(len(listdir(DALLE_NUM)), len(liste_1))

liste_NUM = os.listdir(DALLE_NUM)
liste_ECA = os.listdir(DALLE_ECA)
liste_POS = os.listdir(DALLE_POS)
print(len(liste_NUM), len(liste_ECA)/2, len(liste_POS)/2)

21421 0
21421 10711.0 10681.0


In [24]:
fich_lamp = DATA_DIR+"LampadairePresence_0_1_5m.asc"
header = pd.read_csv(fich_lamp, sep=" ", header=None, skiprows=0, nrows=5, names=["noms","valeurs"])
n_row =  header['valeurs'][0]
n_col =  header['valeurs'][1]
xlow = header['valeurs'][2]
ylow = header['valeurs'][3]
cell_size = header['valeurs'][4]
#ylow = pd.read_csv(fich_lamp, sep=" ", skiprows=3, header=None, nrows=1)[1]
print("Bord inférieur gauche:", xlow, ylow, n_row, n_col, cell_size)

lampadaires = pd.read_csv(fich_lamp, sep=" ", header=None, skiprows=6)
#print(lampadaires.count())
print(lampadaires[0].value_counts())
print(lampadaires.columns)
#print(lampadaires.nunique())
print(lampadaires.nunique().agg(['mean','count','sum']))

# print(lampadaires.describe())
lampes = pd.DataFrame(np.reshape(lampadaires.to_numpy(), n_row*n_col))
print(lampes.shape)

Bord inférieur gauche: 2042500 7272000 200 200 5
0.0    197
1.0      3
Name: 0, dtype: int64
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            190, 191, 192, 193, 194, 195, 196, 197, 198, 199],
           dtype='int64', length=200)
mean       1.79
count    200.00
sum      358.00
dtype: float64
(40000, 1)


In [39]:
lampes

,present,Num0,Num,Num1,Num2,X,Y,nom_ECA,nom_POS
319,1.0,319,119,0.600,1.0,2042505.0,7272595,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
349,1.0,349,149,0.750,1.0,2042505.0,7272745,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
696,1.0,696,96,2.485,3.0,2042515.0,7272480,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
745,1.0,745,145,2.730,3.0,2042515.0,7272725,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
922,1.0,922,122,3.615,4.0,2042520.0,7272610,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
...,...,...,...,...,...,...,...,...,...
38410,1.0,38410,10,191.055,192.0,2043460.0,7272050,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
38606,1.0,38606,6,192.035,193.0,2043465.0,7272030,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
38614,1.0,38614,14,192.075,193.0,2043465.0,7272070,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
38802,1.0,38802,2,193.015,194.0,2043470.0,7272010,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...


In [32]:
lampes['Num0'] = lampes.index
lampes['Num'] = lampes['Num0'].mod(other=n_col)
lampes['Num1'] = (lampes['Num0'] - n_row + 1) / n_row
lampes['Num2'] = lampes['Num1'].apply(np.ceil)

lampes.rename(columns={0: 'present'}, inplace=True)
#lampes['X'] = xlow + np.round((lampes['Num0'] - n_row + 1)/n_row, 0) * cell_size
lampes['X'] = xlow + lampes['Num2'] * cell_size
lampes['Y'] = ylow + lampes['Num'] * cell_size
#lampes['X'] = xlow + lampes.index + (n_row - lampes.index) * n_col
#print(lampes.describe(include='all'))
print(lampes.sum())


present                                                  357
Num0                                                 7374698
Num                                                    37698
Num1                                                 36518.3
Num2                                                   36685
X                                                7.29356e+08
Y                                                 2596292490
nom_ECA    319      2042505.0\n349      2042505.0\n696   ...
nom_POS    319      2042505.0\n349      2042505.0\n696   ...
dtype: object


In [33]:
print(lampes.shape)
lampes = lampes[lampes['present'] > 0]
print(lampes.describe(include='all'))
lampes['nom_ECA'] = str(lampes['X']) + "_" + str(lampes['Y']) + "_EcartZ5m.tif"
lampes['nom_POS'] = str(lampes['X']) + "_" + str(lampes['Y']) + "_Position5m.tif"

print(lampes.shape)

(357, 9)
        present          Num0         Num        Num1        Num2  \
count     357.0    357.000000  357.000000  357.000000  357.000000   
unique      NaN           NaN         NaN         NaN         NaN   
top         NaN           NaN         NaN         NaN         NaN   
freq        NaN           NaN         NaN         NaN         NaN   
mean        1.0  20657.417367  105.596639  102.292087  102.759104   
std         0.0   8581.318801   58.157089   42.906594   42.982597   
min         1.0    319.000000    0.000000    0.600000    1.000000   
25%         1.0  16135.000000   55.000000   79.680000   80.000000   
50%         1.0  22931.000000  120.000000  113.660000  114.000000   
75%         1.0  26943.000000  151.000000  133.720000  134.000000   
max         1.0  39414.000000  198.000000  196.075000  197.000000   

                   X             Y  \
count   3.570000e+02  3.570000e+02   
unique           NaN           NaN   
top              NaN           NaN   
freq      

In [31]:
lampes

,present,Num0,Num,Num1,Num2,X,Y,nom_ECA,nom_POS
319,1.0,319,119,0.600,1.0,2042505.0,7272595,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
349,1.0,349,149,0.750,1.0,2042505.0,7272745,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
696,1.0,696,96,2.485,3.0,2042515.0,7272480,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
745,1.0,745,145,2.730,3.0,2042515.0,7272725,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
922,1.0,922,122,3.615,4.0,2042520.0,7272610,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
...,...,...,...,...,...,...,...,...,...
38410,1.0,38410,10,191.055,192.0,2043460.0,7272050,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
38606,1.0,38606,6,192.035,193.0,2043465.0,7272030,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
38614,1.0,38614,14,192.075,193.0,2043465.0,7272070,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...
38802,1.0,38802,2,193.015,194.0,2043470.0,7272010,319 2042505.0\n349 2042505.0\n696 ...,319 2042505.0\n349 2042505.0\n696 ...


In [37]:
lampes.

,present,Num0,Num,Num1,Num2,X,Y
count,357.0,357.000000,357.000000,357.000000,357.000000,3.570000e+02,3.570000e+02
mean,1.0,20657.417367,105.596639,102.292087,102.759104,2.043014e+06,7.272528e+06
std,0.0,8581.318801,58.157089,42.906594,42.982597,2.149130e+02,2.907854e+02
min,1.0,319.000000,0.000000,0.600000,1.000000,2.042505e+06,7.272000e+06
25%,1.0,16135.000000,55.000000,79.680000,80.000000,2.042900e+06,7.272275e+06
50%,1.0,22931.000000,120.000000,113.660000,114.000000,2.043070e+06,7.272600e+06
75%,1.0,26943.000000,151.000000,133.720000,134.000000,2.043170e+06,7.272755e+06
max,1.0,39414.000000,198.000000,196.075000,197.000000,2.043485e+06,7.272990e+06


In [28]:
lampes.head(2)
lampes.nunique()


present      1
Num0       357
Num        158
Num1       357
Num2       146
X          146
Y          158
nom_ECA      1
nom_POS      1
dtype: int64

In [38]:
# Read files in subdirectories
#liste_f = ["liste_1", "liste_2", "liste_3"]
for n in range(0,len(liste_1)):
#for n in range(0,1):
    #for i in range(0,len(liste_f)):
    #with open(liste_f[n]) as fichierNum:
    with open(liste_3[n]):
        fichierNum3 = liste_3[n]
        df_3a = imageio.imread(fichierNum3)
        df_3 = pd.DataFrame(df_3a)
    print(df_3a.shape, "\n", df_3)
    
    with open(liste_1[n]):         # as fichierNum1:
        fichierNum1 = liste_1[n]
        df_1a = imageio.imread(fichierNum1)
        df_1 = pd.DataFrame(df_1a)
    print(df_1a.shape, "\n", df_1)
    with open(liste_2[n]):         # as fichierNum2:
        fichierNum2 = liste_2[n]
        print(fichierNum2)
        df_2a = imageio.imread(fichierNum2)
        df_2 = pd.DataFrame(df_2a)
    print(df_2a.shape, "\n", df_2)
